In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/DL Lab/Lab7/code')

Mounted at /content/drive


In [ ]:
import time
import torch.backends.cudnn as cudnn
from torch import nn
from easydict import EasyDict as edict
from models import Generator, Discriminator, TruncatedVGG19
from datasets import SRDataset
from utils import *
from solver import train

### Alternating training

In [ ]:
# config
config = edict()
config.csv_folder = '/content/drive/MyDrive/Colab Notebooks/DL Lab/Lab7/data'
config.HR_data_folder = '/content/drive/MyDrive/Colab Notebooks/DL Lab/Lab7/data/DIV2K_train_HR'
config.LR_data_folder = '/content/drive/MyDrive/Colab Notebooks/DL Lab/Lab7/data/DIV2K_train_LR_bicubic/X4'
config.crop_size = 96
config.scaling_factor = 4

# Generator parameters
config.G = edict()
config.G.large_kernel_size = 9
config.G.small_kernel_size = 3
config.G.n_channels = 64
config.G.n_blocks = 16

# Discriminator parameters
config.D = edict()
config.D.kernel_size = 3
config.D.n_channels = 64
config.D.n_blocks = 8
config.D.fc_size = 1024

# Learning parameters
config.checkpoint = '/content/drive/MyDrive/Colab Notebooks/DL Lab/Lab7/code/saved_checkpoints/checkpoint_srgan.pth.tar' # path to model (SRGAN) checkpoint, None if none
config.batch_size = 64
config.start_epoch = 46
config.epochs = 200
config.workers = 4
config.vgg19_i = 5  # the index i in the definition for VGG loss; see paper
config.vgg19_j = 4  # the index j in the definition for VGG loss; see paper
config.beta = 1e-3  # the coefficient to weight the adversarial loss in the perceptual loss
config.print_freq = 100
config.lr = 1e-4

# Default device
config.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cudnn.benchmark = True

In [ ]:
if config.checkpoint is None:
    # Generator
    generator = Generator(config)

    # Initialize generator's optimizer
    optimizer_g = torch.optim.Adam(params=filter(lambda p: p.requires_grad, generator.parameters()),
                                   lr=config.lr)

    # Discriminator
    discriminator = Discriminator(config)
    optimizer_d = torch.optim.Adam(params=filter(lambda p: p.requires_grad, discriminator.parameters()),
                                   lr=config.lr)

else:
    checkpoint = torch.load(config.checkpoint)
    config.start_epoch = checkpoint['epoch'] + 1
    generator = checkpoint['generator']
    discriminator = checkpoint['discriminator']
    optimizer_g = checkpoint['optimizer_g']
    optimizer_d = checkpoint['optimizer_d']
    print("\nLoaded checkpoint from epoch %d.\n" % (checkpoint['epoch'] + 1))


Loaded checkpoint from epoch 107.



In [ ]:
# Truncated VGG19 network to be used in the loss calculation
truncated_vgg19 = TruncatedVGG19(i=config.vgg19_i, j=config.vgg19_j)
truncated_vgg19.eval()

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

TruncatedVGG19(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 

In [ ]:
# Loss functions
content_loss_criterion = nn.MSELoss()
adversarial_loss_criterion = nn.BCEWithLogitsLoss()

In [ ]:
# Move to default device
generator = generator.to(config.device)
discriminator = discriminator.to(config.device)
truncated_vgg19 = truncated_vgg19.to(config.device)
content_loss_criterion = content_loss_criterion.to(config.device)
adversarial_loss_criterion = adversarial_loss_criterion.to(config.device)

In [ ]:
# Custom dataloaders
train_dataset = SRDataset(split='train', config=config)
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=config.batch_size,
                                           shuffle=True, 
                                           num_workers=config.workers,
                                           pin_memory=True)

In [ ]:
# Epochs
for epoch in range(config.start_epoch, config.epochs):
    # At the halfway point, reduce learning rate to a tenth
    if epoch == int(config.epochs / 2 + 1):
        adjust_learning_rate(optimizer_g, 0.1)
        adjust_learning_rate(optimizer_d, 0.1)
    # One epoch's training
    train(train_loader=train_loader,
          generator=generator,
          discriminator=discriminator,
          truncated_vgg19=truncated_vgg19,
          content_loss_criterion=content_loss_criterion,
          adversarial_loss_criterion=adversarial_loss_criterion,
          optimizer_g=optimizer_g,
          optimizer_d=optimizer_d,
          epoch=epoch,
          device=config.device,
          beta=config.beta,
          print_freq=config.print_freq)
    # Save checkpoint
    torch.save({'epoch': epoch,
                'generator': generator,
                'discriminator': discriminator,
                'optimizer_g': optimizer_g,
                'optimizer_d': optimizer_d},
                'checkpoint_srgan.pth.tar')

Epoch: [107][0/1250]----Batch Time 78.207 (78.207)----Data Time 74.996 (74.996)----Cont. Loss 0.3905 (0.3905)----Adv. Loss 0.6931 (0.6931)----Disc. Loss 1.3863 (1.3863)
Epoch: [107][100/1250]----Batch Time 2.630 (3.838)----Data Time 2.269 (3.443)----Cont. Loss 0.4428 (0.4699)----Adv. Loss 0.6931 (0.6931)----Disc. Loss 1.3863 (1.3863)
Epoch: [107][200/1250]----Batch Time 2.835 (3.019)----Data Time 2.468 (2.637)----Cont. Loss 0.4475 (0.4710)----Adv. Loss 0.6931 (0.6931)----Disc. Loss 1.3863 (1.3863)
Epoch: [107][300/1250]----Batch Time 1.242 (2.730)----Data Time 0.875 (2.353)----Cont. Loss 0.4835 (0.4721)----Adv. Loss 0.6931 (0.6931)----Disc. Loss 1.3863 (1.3863)
Epoch: [107][400/1250]----Batch Time 2.344 (2.593)----Data Time 1.977 (2.218)----Cont. Loss 0.4346 (0.4714)----Adv. Loss 0.6931 (0.6931)----Disc. Loss 1.3863 (1.3863)
Epoch: [107][500/1250]----Batch Time 0.911 (2.508)----Data Time 0.543 (2.135)----Cont. Loss 0.5329 (0.4710)----Adv. Loss 0.6931 (0.6931)----Disc. Loss 1.3863 (1.38

In [ ]:
!cp -f /content/checkpoint_srgan.pth.tar /content/drive/MyDrive/Colab\ Notebooks/DL\ Lab/Lab7/code/saved_checkpoints